# Lab 6 - CART and Random Forest Predictions

In this final part of the project, you will build and validate both CART and Random Forest models for predicting each target (Secchi Depth and Total Phosphorus) using the parcel features.

## Tasks

### Data Splitting Strategy

We want to extract 30% of the lakes to be used as a validation set, with the remaining lakes separated into a train/test split (again use 70%/30% split). In summary we should have a random split with:

- **30% of the lakes** marked as "Validation"
- **50% of the lakes** (~70% of 70%) marked as "Training"
- **20% of the lakes** (~30% of 70%) marked as "Test"

### Model Building and Validation

For each target:

1. Perform a grid search for both CART and Random Forests using the training lakes
2. Compare the best model of each type on the test lakes to determine the best overall model
3. Refit the best model on the 70% of the lakes not in the validation set (training + test)
4. Use the validation set to estimate the performance of this model
5. Write up a summary of what we learn

## Step 1: Load and Prepare Data

### Task 1.1: Import Libraries and Load Data

In [1]:
# Import required libraries
%pip install pyarrow
import polars as pl
import polars.selectors as cs
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import pyarrow as pa

Note: you may need to restart the kernel to use updated packages.


In [2]:
(lake_year_data := pl.read_csv('./data/final_dataset_with_parcel_features.csv'))

DNR_ID_Site_Number,LAKE_NAME,Year,latitude,longitude,avg_secchi_depth,avg_total_phosphorus,parcel_count_within_500m,parcel_count_between_501_1600m,emv_land_mean_within_500m,emv_land_mean_between_501_1600m,emv_bldg_mean_within_500m,emv_bldg_mean_between_501_1600m,emv_total_mean_within_500m,emv_total_mean_between_501_1600m,acres_poly_mean_within_500m,acres_poly_mean_between_501_1600m,acres_deed_mean_within_500m,acres_deed_mean_between_501_1600m,fin_sq_ft_mean_within_500m,fin_sq_ft_mean_between_501_1600m,total_tax_mean_within_500m,total_tax_mean_between_501_1600m,tax_capac_mean_within_500m,tax_capac_mean_between_501_1600m,emv_land_median_within_500m,emv_land_median_between_501_1600m,emv_bldg_median_within_500m,emv_bldg_median_between_501_1600m,emv_total_median_within_500m,emv_total_median_between_501_1600m,acres_poly_median_within_500m,acres_poly_median_between_501_1600m,acres_deed_median_within_500m,acres_deed_median_between_501_1600m,fin_sq_ft_median_within_500m,fin_sq_ft_median_between_501_1600m,total_tax_median_within_500m,total_tax_median_between_501_1600m,tax_capac_median_within_500m,tax_capac_median_between_501_1600m,emv_land_std_within_500m,emv_land_std_between_501_1600m,emv_bldg_std_within_500m,emv_bldg_std_between_501_1600m,emv_total_std_within_500m,emv_total_std_between_501_1600m,acres_poly_std_within_500m,acres_poly_std_between_501_1600m,acres_deed_std_within_500m,acres_deed_std_between_501_1600m,fin_sq_ft_std_within_500m,fin_sq_ft_std_between_501_1600m,total_tax_std_within_500m,total_tax_std_between_501_1600m,tax_capac_std_within_500m,tax_capac_std_between_501_1600m,basement_prop_within_500m,basement_prop_between_501_1600m,garage_prop_within_500m,garage_prop_between_501_1600m,homestead_prop_within_500m,homestead_prop_between_501_1600m
str,str,i64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""82007700-01""","""Goggins Lake""",2006,45.133035,-92.892832,0.956,0.102,69,137,204921.73913,171323.357664,147011.594203,199991.240876,351933.333333,371314.59854,5.32913,4.568102,10.593043,8.489343,1234.115942,1503.124088,2050.202899,2182.788321,0.0,0.0,205400.0,155400.0,153500.0,210100.0,368600.0,375000.0,0.02,0.23,5.29,5.01,1314.0,1540.0,2366.0,2424.0,0.0,0.0,128945.44089,102820.380406,121999.144809,193184.768868,195013.07925,228000.633558,9.497451,8.09445,11.679387,11.352502,1023.118395,1280.52188,1530.214754,1734.102317,0.0,0.0,0.0,0.0,0.0,0.0,0.811594,0.89781
"""19002700-01""","""Crystal Lake""",2014,44.722968,-93.270366,1.96,0.0244,1765,92,110654.787535,84493.478261,159819.320113,189930.434783,270474.107649,274423.913043,0.430295,0.606739,0.0,0.0,2228.563173,2573.119565,3317.18187,4132.423913,2401.149008,2552.086957,61200.0,61000.0,159500.0,175400.0,238100.0,235900.0,0.32,0.31,0.0,0.0,2188.0,2278.5,2754.0,2792.5,1982.0,2022.0,138105.254957,120359.683298,82338.486478,129628.188275,168611.758605,230124.674938,0.769999,1.731464,0.0,0.0,1129.582736,2101.404391,3914.574714,9548.042251,1966.305293,2962.754219,0.0,0.0,0.0,0.0,0.850425,0.858696
"""82012200-01""","""Pine Tree Lake""",2004,45.102314,-92.953869,2.2625,0.027375,273,424,173076.556777,160262.971698,301198.168498,319631.839623,474274.725275,479894.811321,2.584652,2.91342,5.146777,6.124481,0.0,0.0,3770.131868,3905.820755,0.0,0.0,185000.0,150000.0,267300.0,283000.0,477300.0,447600.0,0.07,0.205,2.63,2.895,0.0,0.0,3632.0,3458.0,0.0,0.0,111947.950582,135619.676884,332707.974039,260368.938186,398363.670937,309997.671648,7.763072,10.417772,10.391988,15.262668,0.0,0.0,3475.142043,2891.66139,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
"""10001900-01""","""Bavaria Lake""",2013,44.838122,-93.637789,1.2,0.034636,1042,2547,109553.071017,72883.078131,180958.733205,175628.857479,290511.804223,248511.935611,1.198081,1.025014,0.287226,0.597275,1842.429942,1718.336867,3984.12476,3252.698076,0.0,0.0,95000.0,68

In [4]:
# Split lakes: 30% validation, 50% training, 20% test
np.random.seed(42)
(unique_lakes := lake_year_data['DNR_ID_Site_Number'].unique().to_list())
np.random.shuffle(unique_lakes)

(val_split := int(0.30 * len(unique_lakes)))
(train_split := val_split + int(0.5 * len(unique_lakes)))

(lake_year_data := lake_year_data.with_columns(
    pl.when(pl.col('DNR_ID_Site_Number').is_in(unique_lakes[:val_split]))
      .then(pl.lit('Validation'))
    .when(pl.col('DNR_ID_Site_Number').is_in(unique_lakes[val_split:train_split]))
      .then(pl.lit('Training'))
    .otherwise(pl.lit('Test'))
    .alias('split')
))

(lake_year_data
 .group_by('split')
 .agg(pl.len().alias('rows'))
 .with_columns(
     (pl.col('rows') / pl.col('rows').sum()).alias('proportion')
 )
 .sort('split')
)

split,rows,proportion
str,u32,f64
"""Test""",110,0.212766
"""Training""",253,0.489362
"""Validation""",154,0.297872


### Train Test Split

In [19]:
# Identify columns to exclude (IDs, target variables, split column, and any string columns)
exclude_cols = ['DNR_ID_Site_Number', 'Monit_MAP_CODE1', 'Year', 
                'avg_secchi_depth', 'avg_total_phosphorus', 'Avg_Secchi', 'Avg_Total_Phosphorus',
                'Lake_Name', 'lake_name', 'centroid_lat', 'centroid_long', 'split']

# Get feature columns (only numeric columns)
feature_cols = [col for col in lake_year_data.columns 
                if col not in exclude_cols and lake_year_data[col].dtype in [pl.Int64, pl.Float64]]

# Split into training+test (70%) and validation (30%)
(validation_df := lake_year_data.filter(pl.col('split') == 'Validation'))

# Further split training+test into train and test
(train_df := train_test_df.filter(pl.col('split') == 'Training'))
(test_df := train_test_df.filter(pl.col('split') == 'Test'))

DNR_ID_Site_Number,LAKE_NAME,Year,latitude,longitude,avg_secchi_depth,avg_total_phosphorus,parcel_count_within_500m,parcel_count_between_501_1600m,emv_land_mean_within_500m,emv_land_mean_between_501_1600m,emv_bldg_mean_within_500m,emv_bldg_mean_between_501_1600m,emv_total_mean_within_500m,emv_total_mean_between_501_1600m,acres_poly_mean_within_500m,acres_poly_mean_between_501_1600m,acres_deed_mean_within_500m,acres_deed_mean_between_501_1600m,fin_sq_ft_mean_within_500m,fin_sq_ft_mean_between_501_1600m,total_tax_mean_within_500m,total_tax_mean_between_501_1600m,tax_capac_mean_within_500m,tax_capac_mean_between_501_1600m,emv_land_median_within_500m,emv_land_median_between_501_1600m,emv_bldg_median_within_500m,emv_bldg_median_between_501_1600m,emv_total_median_within_500m,emv_total_median_between_501_1600m,acres_poly_median_within_500m,acres_poly_median_between_501_1600m,acres_deed_median_within_500m,acres_deed_median_between_501_1600m,fin_sq_ft_median_within_500m,fin_sq_ft_median_between_501_1600m,total_tax_median_within_500m,total_tax_median_between_501_1600m,tax_capac_median_within_500m,tax_capac_median_between_501_1600m,emv_land_std_within_500m,emv_land_std_between_501_1600m,emv_bldg_std_within_500m,emv_bldg_std_between_501_1600m,emv_total_std_within_500m,emv_total_std_between_501_1600m,acres_poly_std_within_500m,acres_poly_std_between_501_1600m,acres_deed_std_within_500m,acres_deed_std_between_501_1600m,fin_sq_ft_std_within_500m,fin_sq_ft_std_between_501_1600m,total_tax_std_within_500m,total_tax_std_between_501_1600m,tax_capac_std_within_500m,tax_capac_std_between_501_1600m,basement_prop_within_500m,basement_prop_between_501_1600m,garage_prop_within_500m,garage_prop_between_501_1600m,homestead_prop_within_500m,homestead_prop_between_501_1600m,split
str,str,i64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""10001900-01""","""Bavaria Lake""",2013,44.838122,-93.637789,1.2,0.034636,1042,2547,109553.071017,72883.078131,180958.733205,175628.857479,290511.804223,248511.935611,1.198081,1.025014,0.287226,0.597275,1842.429942,1718.336867,3984.12476,3252.698076,0.0,0.0,95000.0,68800.0,118200.0,160200.0,181800.0,231700.0,0.375,0.24,0.0,0.0,1810.0,1877.0,2396.0,2922.0,0.0,0.0,128616.137746,141142.755588,208778.039194,407088.401661,292861.637464,463765.839322,8.997494,8.072278,1.279927,5.961486,1393.877187,1417.958724,4097.902842,6406.97782,0.0,0.0,0.0,0.0,0.747601,0.709462,0.588292,0.630153,"""Test"""
"""27062700-01""","""Northwood Lake""",2010,45.025563,-93.391715,0.98,0.1369,1875,5222,68345.6,89336.844121,158703.573333,194060.283416,227049.173333,283397.127537,0.481685,0.537135,0.0,0.0,1177.661867,1135.151666,3278.376,5176.414975,2333.544533,3374.308311,69000.0,63000.0,144000.0,149000.0,213000.0,213000.0,0.24,0.24,0.0,0.0,1155.0,1150.0,3024.0,3025.0,2130.0,2137.5,112070.986926,204400.048882,328313.70949,409851.697753,439743.841661,587272.905164,1.244839,2.085727,0.0,0.0,433.938655,493.054281,6881.721048,15629.617448,5505.824908,9262.755279,0.200533,0.144006,0.949333,0.883187,0.9184,0.876867,"""Test"""
"""82009002-01""","""Wilmes Lake""",2007,44.927944,-92.912605,1.263636,0.080727,1255,4067,88831.952191,77066.953528,258181.115538,232763.830834,347013.067729,309830.784362,0.0,0.0,0.471825,0.363664,0.0,0.0,0.0,0.0,0.0,0.0,80000.0,80000.0,257300.0,213700.0,340700.0,294100.0,0.0,0.0,0.31,0.27,0.0,0.0,0.0,0.0,0.0,0.0,60568.894491,107575.864077,90770.955605,565968.589123,104814.448208,652044.332504,0.0,0.0,1.26507,0.889599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Test"""
"""19002200-01""","""Long Lake""",2014,44.75592,-93.173208,1.426232,0.078174,992,4153,71217.237903,72511.172646,177276.915323,195412.087647,248494.153226,267923.260294,0.583347,0.461404,0.0,0.0,2272.517137,2252.638093,3004.555444,3077.391765,2119.890121,2585.45413,66100

### Seperate Response and Predictor Variables

In [20]:
# Prepare X and y for training, test, and validation sets
# Training data
(X_train := train_df.select(feature_cols).to_pandas())
(y_train_secchi := train_df.select('avg_secchi_depth').to_pandas().values.ravel())
(y_train_phosphorus := train_df.select('avg_total_phosphorus').to_pandas().values.ravel())

# Test data
(X_test := test_df.select(feature_cols).to_pandas())
(y_test_secchi := test_df.select('avg_secchi_depth').to_pandas().values.ravel())
(y_test_phosphorus := test_df.select('avg_total_phosphorus').to_pandas().values.ravel())

# Validation data
(X_validation := validation_df.select(feature_cols).to_pandas())
(y_validation_secchi := validation_df.select('avg_secchi_depth').to_pandas().values.ravel())
(y_validation_phosphorus := validation_df.select('avg_total_phosphorus').to_pandas().values.ravel())

# Setup cross-validation
(cv_obj := KFold(n_splits=5, shuffle=True, random_state=42))

KFold(n_splits=5, random_state=42, shuffle=True)

## Step 2: Model Training - Secchi Depth

### Grid Search for CART and Random Forest

In [9]:
# CART Grid Search for Secchi Depth
cart_params = {
    'max_depth': [3, 5, 7, 10, 12],
    'min_samples_split': [7, 10, 20, 30, 50],
    'min_samples_leaf': [5, 10, 20, 30],
    'ccp_alpha': [0.0, 0.01, 0.05]
}

(cart_grid_secchi := GridSearchCV(
    DecisionTreeRegressor(random_state=42), 
    cart_params, 
    cv=cv_obj, 
    scoring='neg_mean_squared_error',
    n_jobs=-1
))

cart_grid_secchi.fit(X_train, y_train_secchi)

(cart_best_score_secchi := (-1 * cart_grid_secchi.best_score_) ** 0.5)
cart_grid_secchi.best_params_

{'ccp_alpha': 0.0,
 'max_depth': 10,
 'min_samples_leaf': 5,
 'min_samples_split': 7}

In [12]:
# Random Forest Grid Search for Secchi Depth
rf_params = {
    'n_estimators': [20, 50, 55, 65, 100],
    'max_depth': [5, 10, 15],
    'min_samples_split': [10, 20, 30],
    'min_samples_leaf': [5, 10, 20],
    'max_features': ['sqrt', 'log2']
}

(rf_grid_secchi := GridSearchCV(
    RandomForestRegressor(random_state=42), 
    rf_params, 
    cv=cv_obj, 
    scoring='neg_mean_squared_error',
    n_jobs=-1
))

rf_grid_secchi.fit(X_train, y_train_secchi)

(rf_best_score_secchi := (-1 * rf_grid_secchi.best_score_) ** 0.5)
rf_grid_secchi.best_params_

{'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 65}

### Compare Models on Test Set

In [13]:
# Evaluate CART on test set
(y_test_pred_cart_secchi := cart_grid_secchi.predict(X_test))
(cart_test_mse_secchi := mean_squared_error(y_test_secchi, y_test_pred_cart_secchi))
(cart_test_rmse_secchi := cart_test_mse_secchi ** 0.5)
(cart_test_r2_secchi := r2_score(y_test_secchi, y_test_pred_cart_secchi))

# Evaluate Random Forest on test set
(y_test_pred_rf_secchi := rf_grid_secchi.predict(X_test))
(rf_test_mse_secchi := mean_squared_error(y_test_secchi, y_test_pred_rf_secchi))
(rf_test_rmse_secchi := rf_test_mse_secchi ** 0.5)
(rf_test_r2_secchi := r2_score(y_test_secchi, y_test_pred_rf_secchi))

# Compare results
pd.DataFrame({
    'Model': ['CART', 'Random Forest'],
    'RMSE': [cart_test_rmse_secchi, rf_test_rmse_secchi],
    'R²': [cart_test_r2_secchi, rf_test_r2_secchi]
})

,Model,RMSE,R²
0,CART,1.691302,-2.079224
1,Random Forest,1.026664,-0.134633


### Refit Best Model on Training + Test, Validate on Validation Set

In [14]:
# Combine training and test sets
(X_train_test := pd.concat([X_train, X_test]))
(y_train_test_secchi := np.concatenate([y_train_secchi, y_test_secchi]))

# Determine best model (lower RMSE is better)
(best_model_type_secchi := 'Random Forest' if rf_test_rmse_secchi < cart_test_rmse_secchi else 'CART')
(best_model_secchi := rf_grid_secchi.best_estimator_ if rf_test_rmse_secchi < cart_test_rmse_secchi else cart_grid_secchi.best_estimator_)

# Refit on combined training + test
best_model_secchi.fit(X_train_test, y_train_test_secchi)

# Evaluate on validation set
(y_val_pred_secchi := best_model_secchi.predict(X_validation))
(val_rmse_secchi := mean_squared_error(y_validation_secchi, y_val_pred_secchi) ** 0.5)
(val_r2_secchi := r2_score(y_validation_secchi, y_val_pred_secchi))

pd.DataFrame({
    'Metric': ['Best Model', 'Validation RMSE', 'Validation R²'],
    'Secchi Depth': [best_model_type_secchi, val_rmse_secchi, val_r2_secchi]
})

,Metric,Secchi Depth
0,Best Model,Random Forest
1,Validation RMSE,0.830873
2,Validation R²,-0.101414


## Step 3: Model Training - Total Phosphorus

### Grid Search for CART and Random Forest

In [15]:
# CART Grid Search for Total Phosphorus
(cart_grid_phosphorus := GridSearchCV(
    DecisionTreeRegressor(random_state=42), 
    cart_params, 
    cv=cv_obj, 
    scoring='neg_mean_squared_error',
    n_jobs=-1
))

cart_grid_phosphorus.fit(X_train, y_train_phosphorus)

(cart_best_score_phosphorus := (-1 * cart_grid_phosphorus.best_score_) ** 0.5)
cart_grid_phosphorus.best_params_

{'ccp_alpha': 0.0,
 'max_depth': 5,
 'min_samples_leaf': 5,
 'min_samples_split': 7}

In [16]:
# Random Forest Grid Search for Total Phosphorus
(rf_grid_phosphorus := GridSearchCV(
    RandomForestRegressor(random_state=42), 
    rf_params, 
    cv=cv_obj, 
    scoring='neg_mean_squared_error',
    n_jobs=-1
))

rf_grid_phosphorus.fit(X_train, y_train_phosphorus)

(rf_best_score_phosphorus := (-1 * rf_grid_phosphorus.best_score_) ** 0.5)
rf_grid_phosphorus.best_params_

{'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 20}

### Compare Models on Test Set

In [17]:
# Evaluate CART on test set
(y_test_pred_cart_phosphorus := cart_grid_phosphorus.predict(X_test))
(cart_test_mse_phosphorus := mean_squared_error(y_test_phosphorus, y_test_pred_cart_phosphorus))
(cart_test_rmse_phosphorus := cart_test_mse_phosphorus ** 0.5)
(cart_test_r2_phosphorus := r2_score(y_test_phosphorus, y_test_pred_cart_phosphorus))

# Evaluate Random Forest on test set
(y_test_pred_rf_phosphorus := rf_grid_phosphorus.predict(X_test))
(rf_test_mse_phosphorus := mean_squared_error(y_test_phosphorus, y_test_pred_rf_phosphorus))
(rf_test_rmse_phosphorus := rf_test_mse_phosphorus ** 0.5)
(rf_test_r2_phosphorus := r2_score(y_test_phosphorus, y_test_pred_rf_phosphorus))

# Compare results
pd.DataFrame({
    'Model': ['CART', 'Random Forest'],
    'RMSE': [cart_test_rmse_phosphorus, rf_test_rmse_phosphorus],
    'R²': [cart_test_r2_phosphorus, rf_test_r2_phosphorus]
})

,Model,RMSE,R²
0,CART,0.082659,-2.424455
1,Random Forest,0.061694,-0.907672


### Refit Best Model on Training + Test, Validate on Validation Set

In [18]:
# Prepare combined training + test for phosphorus
(y_train_test_phosphorus := np.concatenate([y_train_phosphorus, y_test_phosphorus]))

# Determine best model (lower RMSE is better)
(best_model_type_phosphorus := 'Random Forest' if rf_test_rmse_phosphorus < cart_test_rmse_phosphorus else 'CART')
(best_model_phosphorus := rf_grid_phosphorus.best_estimator_ if rf_test_rmse_phosphorus < cart_test_rmse_phosphorus else cart_grid_phosphorus.best_estimator_)

# Refit on combined training + test
best_model_phosphorus.fit(X_train_test, y_train_test_phosphorus)

# Evaluate on validation set
(y_val_pred_phosphorus := best_model_phosphorus.predict(X_validation))
(val_rmse_phosphorus := mean_squared_error(y_validation_phosphorus, y_val_pred_phosphorus) ** 0.5)
(val_r2_phosphorus := r2_score(y_validation_phosphorus, y_val_pred_phosphorus))

pd.DataFrame({
    'Metric': ['Best Model', 'Validation RMSE', 'Validation R²'],
    'Total Phosphorus': [best_model_type_phosphorus, val_rmse_phosphorus, val_r2_phosphorus]
})

,Metric,Total Phosphorus
0,Best Model,Random Forest
1,Validation RMSE,0.052633
2,Validation R²,-1.897082


### These Models are Terrible, I've Reran them a bunch of different ways and I get negative R^2 more often than not. The best I've gotten was .42 for CART. At this point I think it's unlikely that I'm able to make a model that provides any real predictive value on water quality. 

- If there is a way to predict water quality much better it's probably an error in my Lab 5 aggregation step.

##### I'm going to conclude that I am unable to predict phosphorus or secchi depth with the predictors I've come up with.